## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        initializer = tf.keras.initializers.HeNormal()  # He 初始化
        self.W1 = tf.Variable(initializer(shape=(28 * 28, 128)), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(initializer(shape=(128, 10)), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)

        # 额外添加 Batch Normalization 层
        self.batch_norm = tf.keras.layers.BatchNormalization()
        ####################
    
    def __call__(self, x, training=True):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flattened = tf.reshape(x, [-1, 784])

        # 隐藏层
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        h1 = self.batch_norm(h1, training=training)  # 归一化
        activated_h1 = tf.nn.relu(h1)  # 使用 ReLU

        # 输出层,分类任务最后输出的是未归一化的 logits，即模型预测的得分
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [17]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.37935847 ; accuracy 0.9016167
epoch 1 : loss 0.37916914 ; accuracy 0.90171665
epoch 2 : loss 0.37898013 ; accuracy 0.90173334
epoch 3 : loss 0.37879157 ; accuracy 0.90178335
epoch 4 : loss 0.3786033 ; accuracy 0.90183336
epoch 5 : loss 0.37841544 ; accuracy 0.9018667
epoch 6 : loss 0.3782279 ; accuracy 0.9018667
epoch 7 : loss 0.3780407 ; accuracy 0.9019167
epoch 8 : loss 0.3778539 ; accuracy 0.90195
epoch 9 : loss 0.37766746 ; accuracy 0.90201664
epoch 10 : loss 0.37748137 ; accuracy 0.90206665
epoch 11 : loss 0.37729573 ; accuracy 0.9021
epoch 12 : loss 0.37711036 ; accuracy 0.9022
epoch 13 : loss 0.37692532 ; accuracy 0.9022
epoch 14 : loss 0.37674066 ; accuracy 0.9022
epoch 15 : loss 0.37655637 ; accuracy 0.9022833
epoch 16 : loss 0.3763725 ; accuracy 0.9023
epoch 17 : loss 0.37618896 ; accuracy 0.90235
epoch 18 : loss 0.37600574 ; accuracy 0.90236664
epoch 19 : loss 0.37582296 ; accuracy 0.90241665
epoch 20 : loss 0.37564054 ; accuracy 0.90245
epoch 21 : loss 0.37